# Data loading

In [ ]:
import java.io.File

val tensorsFiles = new File("sample_tensors_HaTen").listFiles
    .map(_.getName)
    .filter(f => f.startsWith("tensor") && !f.contains("clusters")).toList

# Run CP

In [ ]:
import scala.collection.mutable.{Map => MMap}
import scala.sys.process._

In [ ]:
def appendToEnv(key: String, value: String) = scala.util.Properties.envOrNone(key) match {
    case Some(v) if v.nonEmpty => s"$v${System getProperty "path.separator"}$value"
    case _ => value
}

In [ ]:
val hadoopPath = ""
var timeCPALSBigTensor = MMap[Int, MMap[Int, MMap[Double, Int]]]()
val outputPath = "output"

for (dimension <- 3 to 3;
     size <- List(100, 1000, 10000, 100000);
     sparsity <- List(1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10).reverse) {
    val fileName = s"tensor_${dimension}_${size}_${sparsity}.csv"
    if (tensorsFiles.contains(fileName)) {
        println(fileName)
        val nbIterations = 1
        var endTime = 0
        try {
            s"$hadoopPath/bin/hadoop fs -put sample_tensors_HaTen/$fileName ." !!
        } catch {
            case t: Throwable => println(t)
        }
        
        for (j <- 0 until nbIterations) {   
            val dimensionString = (for (_ <- 0 until dimension) yield size).mkString(":")
            val arguments = Array[String](dimensionString, "3", "40", "5", fileName, outputPath).mkString(" ")
            println(s"./BigTensor_run_parafacnn.sh $arguments")
            
            println(java.util.Calendar.getInstance().getTime())
            val startTime = System.currentTimeMillis()
            
            val output = Process(s"./BigTensor_run_parafacnn.sh $arguments", 
                                 new java.io.File("lib"), 
                                 "PATH" -> appendToEnv("PATH", s"$hadoopPath/bin")) ! ProcessLogger(_ => {}, _ => {})
            
            endTime += (System.currentTimeMillis() - startTime).toInt

            println("Execution time: " + (endTime / 1000) + "s")
        }
        val finalTime = (endTime / nbIterations).toInt
        var dimMap = timeCPALSBigTensor.getOrElse(dimension, MMap[Int, MMap[Double, Int]]())
        var sizeMap = dimMap.getOrElse(size, MMap[Double, Int]())
        sizeMap = sizeMap + (sparsity -> finalTime)
        dimMap = dimMap + (size -> sizeMap)
        timeCPALSBigTensor(dimension) = dimMap
    } 
}

# Results

## Export to CSV

In [ ]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.6`
import com.github.tototoshi.csv._

In [ ]:
val f = new java.io.File(s"""results/benchmarkBigTensor.csv""")
val fileExists = f.exists()
val writer = CSVWriter.open(f, append = true)
if (!fileExists) {
    writer.writeRow(List[String]("dimension", "size", "sparsity", "time"))
}
for ((dimension, r1) <- timeCPALSBigTensor; (size, r2) <- r1; (sparsity, time) <- r2) {
    println(List[Any](dimension, size, sparsity, time))
    writer.writeRow(List[Any](dimension, size, sparsity, time))
}
writer.close()